In [1]:
import pandas as pd

In [26]:
df = pd.read_csv("./Track_1.csv")

In [27]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [28]:
#Renameing Coloumn Name For Convinence
df.rename(columns = {'Number of people who will provide maintenance':'NPPM','Purpose of taking loan': 'Objective',
                     'loan amount taken': 'Amount','Guarantor or Debtor': 'Guarantor','Loan History':'LoanStatus',
                     'Number of years of employment': 'Experience','Marital Status' : 'M_Status','Number of loans taken from current bank': 'ExistingLoan',
                     'Age of the applicant in Number of Years': 'Age','amount in current account': 'CA_Balance',
                     'amount in savings account': 'SA_Balance','% of income paid as installment': 'PI_Payment',
                     'Working abroad or not': 'WorkAB', 'Is there telephone number available': 'PhN',
                     'time duration for loan': 'Tenure', 'Owned property': 'prop', 'Type of job performed': 'JobTyp',
                     'Type of Housing':'HouseT','Number of years of stay in current address': 'NOE','Loan Defaulted or not': 'prediction'}, inplace = True)

In [30]:
df.head()

,NPPM,LoanStatus,Objective,Amount,Guarantor,Experience,M_Status,ExistingLoan,Age,CA_Balance,...,Other loans plans taken,WorkAB,PhN,Tenure,prop,JobTyp,HouseT,NOE,prediction,1=defaulted
0,1,0,0.0,1028,0,0.0,0,2,36,0,...,none,1,0,18,0,0,0,3,1,2=not defaulted
1,1,1,1.0,10222,0,1.0,1,1,37,0,...,stores,1,1,48,1,0,0,3,1,NaN
2,1,1,1.0,701,0,0.0,2,1,40,1,...,none,1,0,12,0,1,0,2,1,NaN
3,1,1,1.0,1670,0,2.0,0,1,22,3,...,none,1,1,9,1,0,0,2,2,NaN
4,1,0,0.0,1163,0,0.0,1,1,44,0,...,none,1,1,12,0,0,0,4,1,NaN


In [24]:
df = df.dropna()

In [29]:
df.Objective = df.Objective.replace({"New Car Purchase": 0, "Purchase of radio/television" : 1,'Purchase of furniture/equipment' : 2,'Old Car Repair': 3,'Education Loan': 4,
                                    'Loan for business establishment': 5,'Other repairs': 6,'Purchase of domestic appliances': 7,'Loan for retraining': 8})

df.Guarantor = df.Guarantor.replace({'none' : 0,'gaurantor' : 1,'co-applicant':  2})

df.LoanStatus = df.LoanStatus.replace({'critical account/other loans existing (not at this bank)' : 0,'existing loans paid back duly till now' : 1,'all loans at this bank paid back duly':  2,'delay in paying off loans in the past' : 3,
                                       'no loans taken/all loans paid back duly': 4})

df.Experience = df.Experience.replace({'between 1 and 4 years': 0, 'greater than 4 years': 1,'less than a year': 2,'unemployed': 3})

df.M_Status = df.M_Status.replace({'female and divorced/seperated/married': 0,'male and single': 1,'male and married/widowed': 2,
                                  'male and divorced/seperated': 3})

df.CA_Balance = df.CA_Balance.replace({'no current account': 0,'less than 0': 1,'between 0 and 200': 3,'greater than 200': 4})

df.SA_Balance = df.SA_Balance.replace({'less than 100': 0,'no savings account': 1,'between 500 and 1000': 2,
                                      'between 100 and 500': 3,'greater than 1000': 4})

df.WorkAB = df.WorkAB.replace({'Yes': 1,'No': 0})

df.PhN = df.PhN.replace({'Yes': 1,'No': 0})

df.prop = df.prop.replace({'Real Estate': 0,'car or other property': 1,'No property': 2,'building society savings agreement/life insurance': 3})

df.JobTyp = df.JobTyp.replace({'skilled employee / official': 0,'unskilled - resident': 1,'unemployed/ unskilled - non-resident': 2,'management/ self-employed/highly qualified employee/ officer': 3})

df.HouseT = df.HouseT.replace({'own': 0,'for free': 1,'rent': 3})

(441, 20)

In [32]:
df = df.drop(['1=defaulted','Other loans plans taken'],axis=1)

In [39]:
X = df.drop(['prediction'],axis=1)
Y = df['prediction']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [41]:
print("Size of X Train:- " + str(X_train.shape))
print("Size of Y Train:- " + str(y_train.shape))
print("Size of X Test:- " + str(X_test.shape))
print("Size of Y Test:- " + str(y_test.shape))

Size of X Train:- (352, 19)
Size of Y Train:- (352,)
Size of X Test:- (89, 19)
Size of Y Test:- (89,)


In [42]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.constraints import max_norm

Init Plugin
Init Graph Optimizer
Init Kernel


In [43]:
model = Sequential()

# input layer
model.add(Dense(78,  activation='relu'))
model.add(Dropout(0.2))

# hidden layer
model.add(Dense(39, activation='relu'))
model.add(Dropout(0.2))

# hidden layer
model.add(Dense(19, activation='relu'))
model.add(Dropout(0.2))

# output layer
model.add(Dense(units=1,activation='sigmoid'))

# Compile model

model.compile(loss='binary_crossentropy', optimizer='adam')

Metal device set to: Apple M1


2023-01-20 22:08:46.832322: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-20 22:08:46.833087: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [44]:
model.fit(x=X_train, 
          y=y_train,
          epochs=25,
          batch_size=256,
          validation_data=(X_test, y_test),)

2023-01-20 22:09:03.042725: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-01-20 22:09:03.045849: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/25


2023-01-20 22:09:03.490567: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 3s 160ms/step - loss: -63.1071 - val_loss: -121.1850
Epoch 2/25
2/2 [==============================] - 0s 18ms/step - loss: -99.6027 - val_loss: -141.6371
Epoch 3/25
2/2 [==============================] - 0s 16ms/step - loss: -134.3127 - val_loss: -168.5583
Epoch 4/25
2/2 [==============================] - 0s 16ms/step - loss: -186.6306 - val_loss: -207.5827
Epoch 5/25
2/2 [==============================] - 0s 21ms/step - loss: -210.1330 - val_loss: -253.6649
Epoch 6/25
2/2 [==============================] - 0s 17ms/step - loss: -248.9763 - val_loss: -299.6858
Epoch 7/25
1/2 [==============>...............] - ETA: 0s - loss: -303.4404

2023-01-20 22:09:05.899467: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 0s 17ms/step - loss: -327.2665 - val_loss: -349.6114
Epoch 8/25
2/2 [==============================] - 0s 16ms/step - loss: -363.3214 - val_loss: -407.1572
Epoch 9/25
2/2 [==============================] - 0s 17ms/step - loss: -420.9565 - val_loss: -468.4904
Epoch 10/25
2/2 [==============================] - 0s 16ms/step - loss: -460.3882 - val_loss: -533.0504
Epoch 11/25
2/2 [==============================] - 0s 17ms/step - loss: -531.7057 - val_loss: -601.9517
Epoch 12/25
2/2 [==============================] - 0s 16ms/step - loss: -599.5131 - val_loss: -675.2768
Epoch 13/25
2/2 [==============================] - 0s 16ms/step - loss: -679.9286 - val_loss: -752.9377
Epoch 14/25
2/2 [==============================] - 0s 16ms/step - loss: -780.6258 - val_loss: -835.9786
Epoch 15/25
2/2 [==============================] - 0s 16ms/step - loss: -834.8326 - val_loss: -924.6138
Epoch 16/25
2/2 [==============================] - 0s 19ms/step - loss: -925.4

In [45]:
from sklearn.metrics import confusion_matrix, classification_report

In [46]:
pred = model.predict_classes(X_test)

/Users/hemangjiwnani/miniforge3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
2023-01-20 22:09:34.282923: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [47]:


print(confusion_matrix(y_test,pred))
print("\n")
print(classification_report(y_test,pred))



[[57  0]
 [32  0]]


              precision    recall  f1-score   support

           1       0.64      1.00      0.78        57
           2       0.00      0.00      0.00        32

    accuracy                           0.64        89
   macro avg       0.32      0.50      0.39        89
weighted avg       0.41      0.64      0.50        89



/Users/hemangjiwnani/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hemangjiwnani/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hemangjiwnani/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

In [71]:
# create the model
model = Sequential()
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))

# compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# fit the model to the training data
model.fit(X_train, y_train, epochs=1000, batch_size=32)

# evaluate the model on the test data
test_loss = model.evaluate(X_test, y_test)
print("Test loss:", test_loss)

Epoch 1/1000
 1/11 [=>............................] - ETA: 3s - loss: 0.8104

2023-01-20 22:43:19.604269: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 0s 12ms/step - loss: 2371.4883
Epoch 2/1000
11/11 [==============================] - 0s 7ms/step - loss: 1.8729
Epoch 3/1000
11/11 [==============================] - 0s 7ms/step - loss: 1.2251
Epoch 4/1000
11/11 [==============================] - 0s 7ms/step - loss: 4.3812
Epoch 5/1000
11/11 [==============================] - 0s 8ms/step - loss: 1.2227
Epoch 6/1000
11/11 [==============================] - 0s 7ms/step - loss: 1.0633
Epoch 7/1000
11/11 [==============================] - 0s 8ms/step - loss: 1.1499
Epoch 8/1000
11/11 [==============================] - 0s 8ms/step - loss: 1.4259
Epoch 9/1000
11/11 [==============================] - 0s 8ms/step - loss: 3.1485
Epoch 10/1000
11/11 [==============================] - 0s 7ms/step - loss: 3.9955
Epoch 11/1000
11/11 [==============================] - 0s 8ms/step - loss: 1.4680
Epoch 12/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.8180
Epoch 13/1000
11/11 [=============

11/11 [==============================] - 0s 7ms/step - loss: 0.5643
Epoch 102/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.5643
Epoch 103/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.3901
Epoch 104/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.4032
Epoch 105/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.4154
Epoch 106/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.4358
Epoch 107/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.3983
Epoch 108/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.3817
Epoch 109/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.4966
Epoch 110/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.4599
Epoch 111/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.3549
Epoch 112/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.4497
Epoch 113/1000
11/1

11/11 [==============================] - 0s 8ms/step - loss: 0.2211
Epoch 200/1000
11/11 [==============================] - 0s 9ms/step - loss: 0.2176
Epoch 201/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.2270
Epoch 202/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.2196
Epoch 203/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.2289
Epoch 204/1000
11/11 [==============================] - 0s 9ms/step - loss: 0.2269
Epoch 205/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.2338
Epoch 206/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.2274
Epoch 207/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.3151
Epoch 208/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.2253
Epoch 209/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.2253
Epoch 210/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.2848
Epoch 211/1000
11/

...

11/11 [==============================] - 0s 7ms/step - loss: 0.1747
Epoch 690/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1631
Epoch 691/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1711
Epoch 692/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1822
Epoch 693/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1870
Epoch 694/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1834
Epoch 695/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1774
Epoch 696/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1707
Epoch 697/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1631
Epoch 698/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1773
Epoch 699/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1796
Epoch 700/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1767
Epoch 701/1000
11/1

11/11 [==============================] - 0s 9ms/step - loss: 0.1795
Epoch 788/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1691
Epoch 789/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1632
Epoch 790/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1630
Epoch 791/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1739
Epoch 792/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.1783
Epoch 793/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1637
Epoch 794/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.1625
Epoch 795/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1577
Epoch 796/1000
11/11 [==============================] - 0s 9ms/step - loss: 0.1742
Epoch 797/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1652
Epoch 798/1000
11/11 [==============================] - 0s 9ms/step - loss: 0.1645
Epoch 799/1000
11

11/11 [==============================] - 0s 7ms/step - loss: 0.1683
Epoch 886/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1743
Epoch 887/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1776
Epoch 888/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1736
Epoch 889/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1703
Epoch 890/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1619
Epoch 891/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1681
Epoch 892/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1618
Epoch 893/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1678
Epoch 894/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1590
Epoch 895/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1579
Epoch 896/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1915
Epoch 897/1000
11/1

11/11 [==============================] - 0s 8ms/step - loss: 0.1587
Epoch 984/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1601
Epoch 985/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1638
Epoch 986/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1633
Epoch 987/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1616
Epoch 988/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1699
Epoch 989/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1667
Epoch 990/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1799
Epoch 991/1000
11/11 [==============================] - 0s 7ms/step - loss: 0.1618
Epoch 992/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1602
Epoch 993/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1631
Epoch 994/1000
11/11 [==============================] - 0s 8ms/step - loss: 0.1603
Epoch 995/1000
11/1

2023-01-20 22:44:46.171218: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [72]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

Mean Squared Error: 0.18382610480545233
Mean Absolute Error: 0.35181930091943636


2023-01-20 22:44:46.296755: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [78]:
from sklearn.metrics import f1_score

# Define a threshold
threshold = 0.5

# Predict on test data
y_pred = model.predict(X_test)

# Convert predictions to binary labels
y_pred_binary = (y_pred > threshold).astype(int)

# Calculate F1 score
f1 = f1_score(y_test, y_pred_binary)
print("F1 Score:", f1)

F1 Score: 0.7808219178082192


In [ ]:
# Mean Squared Error: 0.2132391544014103
# Mean Absolute Error: 0.41500331712572763

In [80]:
from sklearn.metrics import accuracy_score
# acc = sklearn.metrics.accuracy_score(y_true, y_pred)

In [88]:
acc = accuracy_score(y_test, y_pred_binary)
print(acc)

0.6404494382022472


In [87]:
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
nb.fit(X_train,y_train)
y_head=nb.predict(X_test)
print("Naive Bayes Algoritması başarım sonucu: ",nb.score(X_test,y_test))

Naive Bayes Algoritması başarım sonucu:  0.6966292134831461


In [89]:
from sklearn.svm import SVC
svc=SVC(random_state=1)
svc.fit(X_train,y_train)
y_head=svc.predict(X_test)
print("Destek Vektör Makineleri Algoritması başarım sonucu: ",svc.score(X_test,y_test))

Destek Vektör Makineleri Algoritması başarım sonucu:  0.6404494382022472


In [91]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
abc=AdaBoostClassifier(n_estimators=100, random_state=0)
abc.fit(X_train,y_train)
y_head=abc.predict(X_test)
print("AdaBoosting Classifier Algoritması başarım sonucu: ",abc.score(X_test,y_test))

AdaBoosting Classifier Algoritması başarım sonucu:  0.7640449438202247


In [94]:
from sklearn.neural_network import MLPClassifier
ysa=MLPClassifier(alpha=1, max_iter=1000)
ysa.fit(X_train,y_train)
y_head=ysa.predict(X_test)
print("Yapay Sinir Ağları Algoritması başarım sonucu: ",ysa.score(X_test,y_test))

Yapay Sinir Ağları Algoritması başarım sonucu:  0.6292134831460674


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 441 entries, 0 to 599
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   NPPM          441 non-null    int64  
 1   LoanStatus    441 non-null    int64  
 2   Objective     441 non-null    float64
 3   Amount        441 non-null    int64  
 4   Guarantor     441 non-null    int64  
 5   Experience    441 non-null    float64
 6   M_Status      441 non-null    int64  
 7   ExistingLoan  441 non-null    int64  
 8   Age           441 non-null    int64  
 9   CA_Balance    441 non-null    int64  
 10  SA_Balance    441 non-null    int64  
 11  PI_Payment    441 non-null    int64  
 12  WorkAB        441 non-null    int64  
 13  PhN           441 non-null    int64  
 14  Tenure        441 non-null    int64  
 15  prop          441 non-null    int64  
 16  JobTyp        441 non-null    int64  
 17  HouseT        441 non-null    int64  
 18  NOE           441 non-null    

In [109]:
df1 = pd.read_csv("./Track_1.csv")

In [110]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 23 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   Unnamed: 0                                     600 non-null    int64 
 1   Number of people who will provide maintenance  600 non-null    int64 
 2   Loan History                                   600 non-null    object
 3   Purpose of taking loan                         591 non-null    object
 4   loan amount taken                              600 non-null    int64 
 5   Guarantor or Debtor                            600 non-null    object
 6   Number of years of employment                  447 non-null    object
 7   Marital Status                                 600 non-null    object
 8   Number of loans taken from current bank        600 non-null    int64 
 9   Age of the applicant in Number of Years        600 non-null    in

In [112]:
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
df1 = df1.drop(['1=defaulted','Other loans plans taken'],axis=1)

In [113]:
#Renameing Coloumn Name For Convinence
df1.rename(columns = {'Number of people who will provide maintenance':'NPPM','Purpose of taking loan': 'Objective',
                     'loan amount taken': 'Amount','Guarantor or Debtor': 'Guarantor','Loan History':'LoanStatus',
                     'Number of years of employment': 'Experience','Marital Status' : 'M_Status','Number of loans taken from current bank': 'ExistingLoan',
                     'Age of the applicant in Number of Years': 'Age','amount in current account': 'CA_Balance',
                     'amount in savings account': 'SA_Balance','% of income paid as installment': 'PI_Payment',
                     'Working abroad or not': 'WorkAB', 'Is there telephone number available': 'PhN',
                     'time duration for loan': 'Tenure', 'Owned property': 'prop', 'Type of job performed': 'JobTyp',
                     'Type of Housing':'HouseT','Number of years of stay in current address': 'NOE','Loan Defaulted or not': 'prediction'}, inplace = True)

In [115]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   NPPM          600 non-null    int64 
 1   LoanStatus    600 non-null    object
 2   Objective     591 non-null    object
 3   Amount        600 non-null    int64 
 4   Guarantor     600 non-null    object
 5   Experience    447 non-null    object
 6   M_Status      600 non-null    object
 7   ExistingLoan  600 non-null    int64 
 8   Age           600 non-null    int64 
 9   CA_Balance    600 non-null    object
 10  SA_Balance    600 non-null    object
 11  PI_Payment    600 non-null    int64 
 12  WorkAB        600 non-null    object
 13  PhN           600 non-null    object
 14  Tenure        600 non-null    int64 
 15  prop          600 non-null    object
 16  JobTyp        600 non-null    object
 17  HouseT        600 non-null    object
 18  NOE           600 non-null    int64 
 19  predicti